In [ ]:
# check if IS_MASTER exists, this variable will only exist if it's being called by MASTER notebook.
# if it does not exist, set it to False
try: IS_MASTER
except: IS_MASTER = False
# The code below will only run if it's NOT being called from MASTER notebook
if not IS_MASTER:
    DATA_DIR = './data/temp/' # 
    RAW_FILE = f'./data/CSVFiles/smallData.csv' # use "small data" here
    PROCESSED_FILE = f'{DATA_DIR}processed.pkl' # always use pickle for fast I/O!
    OTHER_PREPROCESS_PARAMETER = ... # e.g. batch size, sliding window size, etc

In [39]:
#Libraries used in the notebook
import csv
import json
import numpy as np
import pandas as pd
import glob
import os
import ntpath
import pickle
import shutil


In [ ]:
#df.to_pickle(PROCESSED_FILE)
print("This is the preprocessing notebook being called from Master Notebook")
